In [3]:
%pip install requests
%pip install bs4
%pip install flask


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from requests import get
from bs4 import BeautifulSoup
from flask import Flask, render_template, request, redirect, url_for


In [5]:
def save_to_file(file_name, jobs):
    file = open(f"{file_name}.csv", "w")
    file.write("Position,Company,Location,URL\n")

    for job in jobs:
        file.write(f"{job['position']}," + f"{job['company']}," +
                   f"{job['location']}," + f"{job['link']}\n")
        file.close()


In [6]:
def remoteok_extract_jobs(keyword):
    results = []
    url = f"https://remoteok.com/remote-{keyword}-jobs"
    request = get(url, headers={"User-Agent": "Kimchi"})
    if request.status_code == 200:
        soup = BeautifulSoup(request.text, "html.parser")
        jobs = soup.find_all("tr", {"class": "job"})
        for job in jobs:
            # ! strip()은 공백을 제거해준다.
            title = job.find("h2", {"itemprop": "title"}).text.strip()
            company = job.find("h3", {"itemprop": "name"}).text.strip()
            link = job.find("a", {"itemprop": "url"})["href"]
            job_data = {
                "title": title,
                "company": company,
                "link": f"https://remoteok.com{link}"
            }
            results.append(job_data)
            print(f"{title} - {company} - {link}")
            # write your ✨magical✨ code here
        return results
    else:
        print("Can't get jobs.")


In [29]:
def weworkremotely_extract_jobs(keyword):
    results = []
    url = f"https://weworkremotely.com/remote-jobs/search?term={keyword}"
    request = get(url, headers={"User-Agent": "Kimchi"})
    if request.status_code == 200:
        soup = BeautifulSoup(request.text, "html.parser")
        jobs = soup.find_all("li", {"class": "feature"})
        for job in jobs:
            title = job.find("span", {"class": "title"}).text.strip()
            company = job.find("span", {"class": "company"}).text.strip()
            link = job.find_all("a")[1]['href']
            
            job_data = {
                "title": title, 
                "company": company,
                "link": f"https://weworkremotely.com{link}"
            }
            
            results.append(job_data)
            print(f"{title} - {company} - {link}")
        return results


In [30]:
weworkremotely_extract_jobs("python")

Senior Python Engineer: Long-term job - 100% remote - Proxify AB - /remote-jobs/proxify-ab-senior-python-engineer-long-term-job-100-remote
Full Stack Software Engineer (React / Python) - Trustworthy - /remote-jobs/trustworthy-full-stack-software-engineer-react-python
Senior Open Source Developer & DevOps (Python, Django, React, AWS/OpenStack) - OpenCraft - /remote-jobs/opencraft-senior-open-source-developer-devops-python-django-react-aws-openstack
Senior Backend Engineer (Python) - TestGorilla - /remote-jobs/testgorilla-senior-backend-engineer-python-10
Python Platform Engineer - Doximity - /remote-jobs/doximity-python-platform-engineer


[{'title': 'Senior Python Engineer: Long-term job - 100% remote',
  'company': 'Proxify AB',
  'link': 'https://weworkremotely.com/remote-jobs/proxify-ab-senior-python-engineer-long-term-job-100-remote'},
 {'title': 'Full Stack Software Engineer (React / Python)',
  'company': 'Trustworthy',
  'link': 'https://weworkremotely.com/remote-jobs/trustworthy-full-stack-software-engineer-react-python'},
 {'title': 'Senior Open Source Developer & DevOps (Python, Django, React, AWS/OpenStack)',
  'company': 'OpenCraft',
  'link': 'https://weworkremotely.com/remote-jobs/opencraft-senior-open-source-developer-devops-python-django-react-aws-openstack'},
 {'title': 'Senior Backend Engineer (Python)',
  'company': 'TestGorilla',
  'link': 'https://weworkremotely.com/remote-jobs/testgorilla-senior-backend-engineer-python-10'},
 {'title': 'Python Platform Engineer',
  'company': 'Doximity',
  'link': 'https://weworkremotely.com/remote-jobs/doximity-python-platform-engineer'}]

In [8]:
app = Flask("JobScrapper")


@app.route("/")
def home():
    return render_template("home.html", name="jeongbin")


@app.route("/hello")
def hello():
    return 'hello!'


@app.route("/search")
def search():
    keyword = request.args.get("keyword")
    if keyword:
        keyword = keyword.lower()
        jobs = remoteok_extract_jobs(keyword)
        return render_template("search.html", keyword=keyword, jobs=jobs)
    else:
        return redirect("/")


In [9]:
app.run(host="0.0.0.0", port=8000)


 * Serving Flask app 'JobScrapper'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.0.194:8000
Press CTRL+C to quit
